In [32]:
from Crypto.Util.number import *
flag = b"R3CTF{LLL_C0pP3RSm17H_Quaternion_D1P_4ND_LLL_@g@in_XD}"
from Crypto.Cipher import AES
from hashlib import md5
import string

p, q, r, x, y = [getPrime(256) for _ in range(3)] + [getPrime(256) << 128 for _ in range(2)]
secret = "".join([choice(string.ascii_letters) for _ in range(77)])
PR.<i, j, k> = QuaternionAlgebra(Zmod(p*q), -x, -y)
print("🎁 :", [p*q] + list(PR([x+y, p+x, q+y, r])^bytes_to_long(777*secret.encode())) + [AES.new(key=md5(secret.encode()).digest(), nonce=b"Tiffany", mode=AES.MODE_CTR).encrypt(flag).hex()])

🎁 : [9179146701312781699176828536776206089522408831979885137804817119605132824670673896777591947510882312771183820299882701673215709151977703193903616420702637, 188706257709485662889897107268939642280152413424908152855562194130538159229344166143895172825675717408926036013540426973122050052311570664470631060866326, 2682712522093551545327045002884555242296600010649692520986985330242254238488174707977608269114146421801908861117953931511928486194314901772151783668459458, 3450486865638869884029607240891787866556930082379406388731244160308196118526545881858756124529382085993846169512275853780392762817972043910244447967967496, 3978613946907291563196945341686358146709099241100401211979238259502207240204268447607153317575767659025152104242438128290935255838611081957683034411586841, 'fb459084099c44b75f2a1c256b604b187ab4877e78ea2b9fc5320471c319f9063428c72002310df82e1a424425189d0dabebe601031a']


In [33]:
c = [p*q] + list(PR([x+y, p+x, q+y, r])^bytes_to_long(777*secret.encode())) + [AES.new(key=md5(secret.encode()).digest(), nonce=b"Tiffany", mode=AES.MODE_CTR).encrypt(flag).hex()]
enc = c[:-1]

######################################################### get r, p+x, q+y
n, a0_, a1_, a2_, a3_ = enc
t1, t2 = a1_*inverse(a3_, n), a2_*inverse(a3_, n)

bits = 10
for i in range(2^bits):
    rh = i << (256-bits)
    K = 2^(384-((256-bits)))
    L = Matrix(ZZ, [
        [K, 0, t1, t2],
        [0, 2^384, rh*t1, rh*t2],
        [0, 0, n, 0],
        [0, 0, 0, n]
    ])
    L = L.LLL()
    res = list(map(abs, L[0]))
    if(abs(res[1]) == 2^384 and len(bin(abs(res[0] // K))[2:]) <= 256-bits):
        if(len(bin(res[2])[2:]) == 384 and len(bin(res[3])[2:]) == 384):
            r = (res[0] // K) + rh
            if(isPrime(r)):
                p_x, q_y = res[2], res[3]
                break

######################################################### coppersmith to get p, q, x, y
from tqdm import *

pl = p_x % 2^128
bits = 7
for i in trange(2^bits):
    pll = pl + (i << 128)
    PR.<aaa> = PolynomialRing(Zmod(n))
    f = 2^(128+bits)*aaa + pll + 2^255
    f = f.monic()
    #res = uni_small_roots(f, X=2^(128-bits-1), beta=0.499, epsilon=0.01)
    res = f.small_roots(X=2^(128-bits-1), beta=0.499, epsilon=0.025)
    if(res != []):
        p = 2^(128+bits)*int(res[0]) + pll + 2^255
        q = n // p
        assert p*q == n
        x = p_x - p
        y = q_y - q
        break

li_p = [i[0] for i in list(factor(p-1))]
li_q = [i[0] for i in list(factor(q-1))]
ellp = li_p[-1]
ellq = li_q[-1]
print(len(bin(max(li_p[:-1]))[2:]), len(bin(max(li_q[:-1]))[2:]))

 94%|███████████████████████████████████████████████████████████████████████████▉     | 120/128 [00:16<00:01,  7.39it/s]


37 28


In [34]:
print(list(factor(p-1)))
print(list(factor(q-1)))

[(2, 4), (5, 1), (6359, 1), (78030424691, 1), (2295202126216837008978652626989273467723189905747503092156937, 1)]
[(2, 2), (151, 1), (761, 1), (172264199, 1), (1272395036149799262058207121218496577799325253210771211721273551, 1)]


In [5]:
cd cado-nfs

/mnt/c/Users/11423/cado-nfs


In [28]:
######################################################### dlp
def log_cado(ell, p, target):
    import subprocess
    command = [
        './cado-nfs.py',
        '-dlp',
        '-ell', str(ell),
        'target=' + str(target),
        str(p),
    ]
    try:
        result = subprocess.run(command, check=True, text=True, capture_output=True)
        return int(result.stdout)
    except:
        pass

def q2m(q):
    a, b, c, d = q
    return Matrix([
        [a, -b*x, -c*y, -d*x*y],
        [b, a,   -d*y,   c*y],
        [c, d*x,  a,    -b*x],
        [d, -c,    b,     a]
    ])

def dlp(mod, ell, p, q, x, y, r):
    Fp = Zmod(mod)
    M = Matrix(Fp, q2m([x+y, p+x, q+y, r]))
    M_ = Matrix(Fp, q2m(enc[1:]))

    ########################### mod ell
    t1 = log_cado(ell, mod, pow(int(M.det()), (mod-1)//ell, mod))
    t1_inv = inverse(t1,ell)
    t2 = log_cado(ell, mod, pow(int(M_.det()), (mod-1)//ell, mod))
    mell = t2*t1_inv % ell
    mr = Fp(pow(int(M_.det()), ell, mod)).log(Fp(pow(int(M.det()), ell, mod)))
    return crt([mr, mell], [(mod-1)//ell, ell])

In [35]:
mp = dlp(p, ellp, p, q, x, y, r)
print(mp)

77565530500969654570664281202693157575372172179429872000108264650554354260959


In [36]:
mq = dlp(q, ellq, p, q, x, y, r)
print(mq)

82014589803612401378064393526170294232099989507975000336643912630887084005593


In [37]:
mpq = crt([mp, mq], [(p-1) // (2^4*3^0), (q-1) // (2*2*3^0)])
M = Matrix(Zmod(n), q2m([x+y, p+x, q+y, r]))
M_ = Matrix(Zmod(n), q2m(enc[1:]))

print(len(bin(mpq)))
print(M.det()^mpq == M_.det())

507
True


In [39]:
c = mpq
mod = lcm([(p-1) // (2^4*3^0), (q-1) // (2*2*3^0)])
nums = 77
L = Matrix(ZZ, nums+2, nums+2)
for i in range(nums):
    L[i,i] = 1
    L[i,-1] = (sum([256^(i+nums*j) for j in range(777)])) % mod
    c = (c - (sum([256^(i+nums*j) for j in range(777)]))*93) % mod

L[-2,-2] = 1
L[-2,-1] = -c
L[-1,-1] = mod
L[:,-1:] *= mod
L = L.BKZ(block_size=30)
res = L[0]
print(res)

sign = res[-2]
secret = ""
res = (sign // abs(sign)) * res[:-2]
secret = ""
for j in res[::-1]:
    secret += chr(93+j)
print(secret)

(-18, -14, 24, 18, 24, -5, 29, 25, 29, -4, 28, -15, -28, -19, -20, 24, 11, 26, 29, -24, -3, 11, 28, 29, -6, 21, -26, 19, 19, -25, 26, -22, 25, -26, -12, 26, 25, -6, -18, 17, 18, -19, 7, -5, 18, 24, -20, -12, -14, 21, -20, -5, -20, -11, -13, 10, -22, 21, -22, -3, 17, -12, -19, -25, 24, 11, 21, -12, 19, 6, 5, 23, 22, -28, 23, -28, 19, 1, 0)
pAtAstbcpQrhuDJQnZGrGgPRIXIrOQIuoXdJonKWvwQCvGwDppCrWzyhZEzwhuIJANyYzvzXuouOK


In [41]:
print(AES.new(key=md5(secret.encode()).digest(), nonce=b"Tiffany", mode=AES.MODE_CTR).decrypt(bytes.fromhex('fb459084099c44b75f2a1c256b604b187ab4877e78ea2b9fc5320471c319f9063428c72002310df82e1a424425189d0dabebe601031a')))

b'R3CTF{LLL_C0pP3RSm17H_Quaternion_D1P_4ND_LLL_@g@in_XD}'
